In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile,join
import numpy as np
    
def load_data():
    file_path=os.getcwd()
    data_path=file_path+'\\'+'data'
    file_names=[".".join(f.split(".")[:-1]) for f in listdir(data_path) if isfile (join(data_path,f))] 
    full_file_names=[f for f in listdir(data_path) if isfile (join(data_path,f))]
    z=0
    files_ready=[]
    for f,r in zip(file_names,full_file_names):
        globals()[f]=pd.read_csv(data_path+'\\'+r,sep=',')        
        files_ready.append(f)
        z+=1
    return print(files_ready)
load_data()

['CSi_exp1(US,IND)', 'EMiCN_0607_1107', 'ESa', 'SRa_0707_1007', 'SRa_exp31', 'SZi 2005 - 0106', 'SZi_1806-2406', 'SZi_1806-2406_exp2', 'SZi_Exp3(1)', 'SZi_exp3']


In [2]:
data = EMiCN_0607_1107.copy()

In [3]:
import pandas as pd
from datetime import timedelta

metrics = ['dau','time_in_game_sum', 'battle_play_sum', 'battle_end_sum', 'rv_imp_sum', 'is_imp_sum']
app_version = '1.0.3' #change app_version
country = 'Japan'
media_source = 'Organic'


def gameinfo(data, day_window = 3, **kwargs):
    #cast day0date and act date to date
    data['day0_date'] = pd.to_datetime(data['day0_date'])
    data['act_date'] = pd.to_datetime(data['act_date'])

    print("Max act_date: ", data['act_date'].max()) #find max act_date
    data = data[data['act_date'] < data['act_date'].max()] #drop max act_date 
    print("Max act_date after: ", data['act_date'].max()) #max act_date after drop
    data = data[data['day0_date']<=data['act_date'].max()-timedelta(days=day_window)] #find range day0_date
    print(" Max day0: ", data['day0_date'].max(), '\n', "Min day0: ", data['day0_date'].min())
    try:
        data = data[data['event_segment'] == 'app_update--FALSE']
        data = data[data['app_version'] == app_version]
        data = data[data['country'] == country]
        data = data[data['media_source'] == media_source]
    except:
        pass
    data_grouped = data[(data['day_diff']<=day_window) & (data['day_diff']>= 0)]
    data_grouped = data_grouped.groupby('day_diff')[metrics].sum()
    data_grouped.loc['Grand Total'] = data_grouped.iloc[:,1:].sum()
    data_grouped.iloc[:-1,1:] = data_grouped.iloc[:-1,1:].div(data_grouped.iloc[:-1,0], axis=0)
    data_grouped.iloc[-1,1:] = data_grouped.iloc[-1,1:]/data_grouped.iloc[0,0]
    data_grouped['dau'] = data_grouped['dau']/data_grouped.iloc[0,0]
    data_grouped = data_grouped.style.format({'dau': "{:.2%}"})
    return data_grouped
print(app_version)
gameinfo(data, day_window = 3)

1.0.3
Max act_date:  2021-07-12 00:00:00
Max act_date after:  2021-07-11 00:00:00
 Max day0:  2021-07-08 00:00:00 
 Min day0:  2021-07-06 00:00:00


,dau,time_in_game_sum,battle_play_sum,battle_end_sum,rv_imp_sum,is_imp_sum
day_diff,,,,,,
0,100.00%,732.091009,13.838346,12.017857,4.715226,2.384398
1,22.65%,791.547041,15.987552,14.087137,5.170124,2.871369
2,12.50%,609.729286,12.872180,10.917293,3.834586,2.330827
3,9.68%,550.664117,12.300971,10.233010,2.786408,2.320388
Grand Total,nan%,1040.902321,20.259398,17.563910,6.635338,3.550752


In [4]:
app_version = '1.0.3' #change app_version
country = 'Japan'
# media_source = 'Organic'
try:
    data = data[data['event_segment'] == 'app_update--FALSE']
    data = data[data['app_version'] == app_version]
    data = data[data['country'] == country]
    data = data[data['media_source'] == media_source]
except:
    pass

max_stage = data.groupby('resettable_device_id_or_app_instance_id')['max_stage_story'].max().reset_index()
max_stage = max_stage.groupby('max_stage_story').agg('count').rename({'resettable_device_id_or_app_instance_id':'dau'}, axis=1)
max_stage.loc['Grand Total'] = max_stage.sum()
user_left = [max_stage.iloc[-1,:] - max_stage.iloc[0,0]]
dau = pd.Series(max_stage.dau)
for row in dau.index[1:-1]:
    user_left.append(user_left[row-1] - dau[row])
user_left = pd.DataFrame(user_left).reset_index(drop=True)
max_stage = pd.concat([max_stage, user_left], axis=1)
max_stage.index.names = ['stage']
max_stage.columns = ['dau','user_left']
max_stage['drop_rate'] = (max_stage.user_left / max_stage.iloc[-1,0]).map("{:.2%}".format)
print("App version: ", app_version)
max_stage

App version:  1.0.3


,dau,user_left,drop_rate
stage,,,
0,984,3229.0,76.64%
1,457,2772.0,65.80%
2,113,2659.0,63.11%
3,103,2556.0,60.67%
4,46,2510.0,59.58%
5,53,2457.0,58.32%
6,239,2218.0,52.65%
7,269,1949.0,46.26%
8,191,1758.0,41.73%
